In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [2]:
# Carregando os dados
salarios = pd.read_csv('salarios.csv', sep=';')
vs_cad_basc = pd.read_csv('vs_cad_basc.csv',
                            parse_dates=[9, 11, 12, 13, 14, 17],
                            index_col=0,
                            dtype={'SEXO' : 'category', 'EST_CIVIL' : 'category', 'GRAU_INSTR' : pd.CategoricalDtype(categories=['ENSINO FUNDAMENTAL', 'ENSINO MEDIO', 'SUPERIOR EM ANDAMENT', 'SUPERIOR GRADUACAO', 'POS-GRADUCAO-ESPEC', 'MESTRADO', 'DOUTORADO'], ordered=True)})
hst_lczc_fun = pd.read_csv('hst_lczc_fun.csv', na_values=["9999-12-31"], parse_dates=[2, 3, 5])
hst_cmss_fuc_fun = pd.read_csv('hst_cmss_fuc_fun.csv', na_values=["9999-12-31"], parse_dates=[2, 3, 5])
cursos = pd.read_csv('cursos.csv', parse_dates=[4])
eventos = pd.read_csv('Eventos.csv', sep=';')

In [3]:
# Excluindo algumas colunas que não importam
vs_cad_basc.drop(columns='DATA_NOMEACAO_215', inplace=True)
salarios = salarios[['cod', 'vr']]

In [4]:
# Quantidade de pessoas que se desligaram do banco
vs_cad_basc['INATIVIDADE_DT_215'].notna().sum()

78

In [5]:
# Quantidade de pessoas que sairam da Diope
(vs_cad_basc['PREF_LOTACAO'] != '7551617774bcd665e4abe990db4f6f83').sum()

127

In [6]:
# Cria a coluna para indicar se o funcionário saiu do banco ou do departamento
vs_cad_basc['SAIU'] = vs_cad_basc.apply(lambda x : 1 if (pd.notna(x['INATIVIDADE_DT_215'])) | (x['PREF_LOTACAO'] != '7551617774bcd665e4abe990db4f6f83') else 0, axis = 1)

In [7]:
# Visualisa as quantidades
vs_cad_basc['SAIU'].value_counts()

0    277
1    194
Name: SAIU, dtype: int64

In [8]:
# Lista de funcionários que saíram
saiu = vs_cad_basc.index[vs_cad_basc['SAIU'] == 1]

In [9]:
# Registro de movimentação dos funcionários que saíram e vemos os eventos que origiraram a entrada deles nos departamento
movimentacao = hst_lczc_fun[(hst_lczc_fun['MATRICULA'].isin(saiu)) &
                            (hst_lczc_fun['CD_PRF_DEPE_LCZC'] == '7551617774bcd665e4abe990db4f6f83')]
eventos_unicos = movimentacao['CD_TIP_EVT'].unique()
# Lista de eventos únicos
eventos[eventos['CD_TIP_EVT'].isin(eventos_unicos)]

,CD_TIP_EVT,TIP_EVT
14,140,REMOCAO - COM DESPACHO
18,144,REMOCAO-UNIF/DESMEMB/REESTRURACAO PREFIXO
27,153,ADICAO NO INTERESSE DO SERVICO
29,155,RETORNO DE ADICAO
33,159,RETORNO COLABORADOR DA DG
48,176,COMPOSICAO TEMPORARIA DE EQUIPE
74,250,RELOCALIZACAO SEM COMISSAO
75,251,RELOCALIZACAO COM COMISSAO
81,305,NOMEACAO - ALCADA DA DG - C/ALT. DE DE
87,311,COMISSIONAMENTO - ALCADA DIPES/GEPES


In [10]:
# Excluimos os eventos temporários [153, 176]
movimentacao = movimentacao[~movimentacao['CD_TIP_EVT'].isin([153, 176])]
# Vemos a data de saída deles do departamento
saida = movimentacao.groupby('MATRICULA')['DT_FIM_LCZC_FUN'].max()
saida.rename('DT_SAIDA', inplace=True)

MATRICULA
00e510ffcd436a7f1a41d8547fa2a10d   2017-07-02
0131cb0ab3a6b2d184c3e72acc47ff02   2020-03-17
014ced666b77b2f41f904f3395a38b6e   2018-10-07
02d3df5a38ecda03effbccdbcea229d4   2018-03-28
08b23fb78a98391ec4d3490f6493a265   2019-02-07
                                      ...    
f9460781684aa2475473f7456b78450c   2018-02-13
f9626f892ca87baa2174b262d2c7693d   2021-05-11
fabf5d60f3d0578a4249ef2ba597940f   2018-03-11
fb2c79458d4f815af0ec186297331a69   2020-01-12
fd93f0be729780f12f96a5ff49aa4f29   2018-01-07
Name: DT_SAIDA, Length: 194, dtype: datetime64[ns]

In [11]:
# Criando uma coluna no vs_cad_basc com a data de saída do funcionário do departamento
vs_cad_basc = vs_cad_basc.merge(saida, how='left', on='MATRICULA')

In [12]:
# Vamos ver a quantidade de cursos que o funcionário fez durante o período dele no departamento

#Substituindo os NaT (funcionários qua ainda não sairam) por uma data recente
dt_saida = vs_cad_basc['DT_SAIDA'].replace({pd.NaT : '2022-10-30'})

# Colocando a data de saída no dataframe
cursos = cursos.merge(dt_saida, how='left', left_on='matricula', right_on='MATRICULA')

In [13]:
# Só nos interessa os cursos que o funcionário fez até a data de saída dele
cursos = cursos[cursos['DT_FIM_CSO'] <= cursos['DT_SAIDA']]

# Excluindo as colunas que não interessam e renomeando.
cursos.drop(columns=['NM_CSO', 'CD_CNH_CSO', 'DT_FIM_CSO', 'DT_SAIDA'], inplace=True)
cursos.rename(columns={'CD_CSO' : 'QTDE_CSO', 'matricula' : 'MATRICULA'}, inplace=True)

# Vamos saber a quantiodade de curso que cada matrícula fez
cursos = cursos.groupby('MATRICULA').count()


In [14]:
vs_cad_basc = vs_cad_basc.merge(cursos, how='left', left_index=True, right_index=True)

In [15]:
# Vamos transformar a quantidade de curso em uma variável categórica
classes = [0, 100, 200, 300, 400, 1000]
labels = ['até 100', '100 a 200', '200 a 300', '300 a 400', '+ de 400']

vs_cad_basc['FAIXA_QTDE_CSO'] = pd.cut(vs_cad_basc['QTDE_CSO'], classes, labels=labels)

In [16]:
# Vamos criar uma váriavel para a geração (baseado na data de nascimento) do funcionário
classes = [0, 1965, 1980, 1996]
labels = ['Baby Boomer', 'X', 'Millennials']
vs_cad_basc['GERACAO'] = pd.cut(vs_cad_basc['NASC_DT_215'].dt.year, classes, labels=labels)

In [17]:
# Vamos ver o salário médio PONDERADO pela quantidade de dias que o funionário ganhou desde 2017
y = hst_cmss_fuc_fun.set_index('MATRICULA')
y['DT_FIM_CMSS_FUN'].replace({pd.NaT : '2022-10-30'}, inplace=True)
y = y.merge(dt_saida, how='left', left_index=True, right_index=True)

#comissoes = y[(y['DT_FIM_CMSS_FUN'] <= y['DT_SAIDA']) & (y['DT_INC_CMSS_FUN'] >= '2017-01-01')]
comissoes = y[(y['DT_FIM_CMSS_FUN'] <= y['DT_SAIDA'])]
comissoes = comissoes.join(salarios.set_index('cod'), on='CD_TIP_CMSS_FUC')
# Vemos a quantidade de dias que cada matrícula ficou na comissão
comissoes['QTDE_DIAS_CMSS'] = comissoes['DT_FIM_CMSS_FUN'] - comissoes['DT_INC_CMSS_FUN']
# Calculando a média ponderada do salário
salario_pond = comissoes.groupby('MATRICULA').apply(lambda x : np.average(x['vr'], weights=x['QTDE_DIAS_CMSS'].dt.days))
salario_pond.rename('SALARIO_MEDIO', inplace=True)

vs_cad_basc = vs_cad_basc.join(salario_pond)

In [18]:
# Vamos deixar apenas as colunas relevantes
vs_cad_basc = vs_cad_basc[['EST_CIVIL', 'SEXO', 'GERACAO', 'GRAU_INSTR', 'FAIXA_QTDE_CSO', 'SALARIO_MEDIO', 'SAIU']]

In [19]:
fig = px.histogram(vs_cad_basc, x='EST_CIVIL', color='SAIU', text_auto=True)
fig.show()

In [20]:
fig = px.histogram(vs_cad_basc, x='SEXO', color='SAIU', text_auto=True)
fig.show()

In [21]:
fig = px.histogram(vs_cad_basc, x='GERACAO', color='SAIU', text_auto=True, category_orders = {'GERACAO' : vs_cad_basc['GERACAO'].cat.categories})
fig.show()

In [22]:
fig = px.histogram(vs_cad_basc, x='GRAU_INSTR', color='SAIU', text_auto=True, category_orders = {'GRAU_INSTR' : vs_cad_basc['GRAU_INSTR'].cat.categories})
fig.show()

In [23]:
fig = px.histogram(vs_cad_basc, x='FAIXA_QTDE_CSO', color='SAIU', text_auto=True, category_orders = {'FAIXA_QTDE_CSO' : vs_cad_basc['FAIXA_QTDE_CSO'].cat.categories})
fig.show()

In [24]:
fig = px.histogram(vs_cad_basc, x='SALARIO_MEDIO', color='SAIU')
fig.show()

In [25]:
vs_cad_basc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 471 entries, 02d3df5a38ecda03effbccdbcea229d4 to b3f77303944eca28d4fbc378414cd344
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   EST_CIVIL       471 non-null    category
 1   SEXO            471 non-null    category
 2   GERACAO         471 non-null    category
 3   GRAU_INSTR      471 non-null    category
 4   FAIXA_QTDE_CSO  471 non-null    category
 5   SALARIO_MEDIO   439 non-null    float64 
 6   SAIU            471 non-null    int64   
dtypes: category(5), float64(1), int64(1)
memory usage: 30.4+ KB


In [26]:
encoder = OneHotEncoder(sparse=False)
data1_encoded = encoder.fit_transform(vs_cad_basc[['EST_CIVIL', 'SEXO']])

columns1_encoded  = encoder.get_feature_names_out(vs_cad_basc[['EST_CIVIL', 'SEXO']].columns)
data1_encoded = pd.DataFrame(data1_encoded, columns=columns1_encoded)


In [27]:
oe = OrdinalEncoder(categories=[
    ['Baby Boomer', 'X', 'Millennials'],
    ['ENSINO FUNDAMENTAL', 'ENSINO MEDIO', 'SUPERIOR EM ANDAMENT', 'SUPERIOR GRADUACAO', 'POS-GRADUCAO-ESPEC', 'MESTRADO', 'DOUTORADO'],
    ['até 100', '100 a 200', '200 a 300', '300 a 400', '+ de 400']])
data2_encoded = oe.fit_transform(vs_cad_basc[['GERACAO', 'GRAU_INSTR', 'FAIXA_QTDE_CSO']])

columns2_encoded = oe.get_feature_names_out(vs_cad_basc[['GERACAO', 'GRAU_INSTR', 'FAIXA_QTDE_CSO']].columns)

data2_encoded = pd.DataFrame(data2_encoded, columns=columns2_encoded)

In [36]:
X = data1_encoded.join(data2_encoded)
y = vs_cad_basc.reset_index()['SAIU']


In [38]:
# Vamos dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

LogisticRegression()

In [45]:
# Usando o modelo de regressão logística para treinar e ver a acurácia
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

print('Acurácia da regressão logística: {:.3f}'.format(accuracy_score(y_test, logreg.predict(X_test))))

Acurácia da regressão logística: 0.732


In [46]:
# Mesma coisa para Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

print('Acurácia da Floresta Aleatória: {:.3f}'.format(accuracy_score(y_test, rf.predict(X_test))))

Acurácia da Floresta Aleatória: 0.739


In [47]:
# Mesma coisa para SVM
clf = svm.SVC()
clf.fit(X_train, y_train)

print('Acurácia da SVM: {:.3f}'.format(accuracy_score(y_test, clf.predict(X_test))))

Acurácia da SVM: 0.754
